The code requires the modeling results file. It should be downloaded from https://github.com/icredd-cheminfo/Photoswitches repository.

## Initial imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
import glob, os, pickle
from sklearn.datasets import load_svmlight_file, dump_svmlight_file
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
from doptools.chem.chem_features import ChythonCircus, Fingerprinter, ComplexFragmentor
from doptools.chem.solvents import SolventVectorizer
from chython import smiles

def r2(a, b):
    return 1. - np.sum((a-b)**2)/np.sum((a-np.mean(a))**2)

def rmse(a, b):
    return np.sqrt(np.sum((a-b)**2)/len(a))

from sklearn.metrics import mean_absolute_error as mae

colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00']

In [ ]:
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Arial']})
#rc('font',**{'family':'serif','serif':['Times']})

In [ ]:
%config IPCompleter.use_jedi = False

# Reading data

In [ ]:
fdata = pd.read_excel('Photoswitch data.xls', sheet_name='Full set')
fdata['mol'] = [smiles(c) for c in fdata['SMILES']]
[c.canonicalize() for c in fdata.mol]
[c.clean2d() for c in fdata.mol]
print()

# Dataset preparation

The code here separate the training and test sets for both absorption maximum and half-life. The separated sets can also be found in the initial Excel file on the separate sheets. The code is reproducible as it uses specific random seeds.

In [ ]:
from numpy import histogram
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split

fdata_log = fdata[pd.notnull(fdata["logt12"])]

kbd_logt12 = KBinsDiscretizer(n_bins=10, encode="ordinal")
a = kbd_logt12.fit_transform(fdata_log[["logt12"]])

logt12_labels = np.ravel(a)

train_logt12, test_logt12 = train_test_split(fdata_log.index, test_size=0.1, stratify=logt12_labels, random_state=10)

plt.hist(fdata_log["logt12"].loc[train_logt12], color=colors[0], edgecolor='k', label="Training set")
plt.hist(fdata_log["logt12"].loc[test_logt12], color=colors[1], edgecolor='k', label="Test set")
plt.ylabel("# of molecules", fontsize=14)
plt.xlabel(r"Thermal half-time log$t_{1/2}$", fontsize=14)
plt.legend(fontsize=14)
print(len(train_logt12), len(test_logt12))

fdata_log.loc[train_logt12].to_excel("fulldata.logt12_train.xlsx")
fdata_log.loc[test_logt12].to_excel("fulldata.logt12_test.xlsx")
#plt.savefig("Histogram logt12.png")

In [ ]:
fdata_lambda = fdata[pd.notnull(fdata["lambda"])]

kbd_lambda = KBinsDiscretizer(n_bins=10, encode="ordinal")
a = kbd_lambda.fit_transform(fdata_lambda[["lambda"]])

lambda_labels = np.ravel(a)

train_lambda, test_lambda = train_test_split(fdata_lambda.index, test_size=0.1, stratify=lambda_labels, random_state=42)

plt.hist(fdata_lambda["lambda"].loc[train_lambda], color=colors[0], edgecolor='k', label="Training set")
plt.hist(fdata_lambda["lambda"].loc[test_lambda], color=colors[1], edgecolor='k', label="Test set")
plt.ylabel("# of molecules", fontsize=14)
plt.xlabel(r"Absorption maximum $\lambda_{max}$", fontsize=14)
plt.legend(fontsize=14)
print(len(train_lambda), len(test_lambda))

fdata_lambda.loc[train_lambda].to_excel("fulldata.lambda_train.xlsx")
fdata_lambda.loc[test_lambda].to_excel("fulldata.lambda_test.xlsx")
#plt.savefig("Histogram lambda.png")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4), facecolor="w")

ax[0].hist(fdata_lambda["lambda"].loc[train_lambda], color=colors[0], edgecolor='k', label="Training set")
ax[0].hist(fdata_lambda["lambda"].loc[test_lambda], color=colors[1], edgecolor='k', label="Test set")
ax[0].set_ylabel("# of molecules", fontsize=14)
ax[0].set_xlabel(r"Absorption maximum $\lambda_{max}$", fontsize=14)
ax[0].legend(fontsize=14)

ax[1].hist(fdata_log["logt12"].loc[train_logt12], color=colors[0], edgecolor='k', label="Training set")
ax[1].hist(fdata_log["logt12"].loc[test_logt12], color=colors[1], edgecolor='k', label="Test set")
ax[1].set_ylabel("# of molecules", fontsize=14)
ax[1].set_xlabel(r"Thermal half-life log$t_{1/2}$", fontsize=14)
ax[1].legend(fontsize=14)
plt.tight_layout(pad=3)
#plt.savefig("Histogram lambda+logt12.png", dpi=300)

# Best models for prediction of $\lambda_{max}$

The code here reads the benchmark results for models predictiong absorption maximum and reconstructs the plots. The example code is given for reconstructing the plots for SVM models, however, Random forest or XGBoost models can also be recreated by changing the folder names (adding RFR or XG in the end for all of them emtries in the 'names' variable).

In [ ]:
names = {'RDKFP':'models/lambda models/Model_lambda_rdkfp',
         'RDKFP layered':'models/lambda models/Model_lambda_layered',
         'ChyLine':'models/lambda models/Model_lambda_chyline',
         'Morgan':'models/lambda models/Model_lambda_morgan',
         'CircuS':'models/lambda models/Model_lambda_circus',
         'RDKFP linear':'models/lambda models/Model_lambda_rdkfplinear',
         'AtomPairs':'models/lambda models/Model_lambda_atompairs',
         'Morgan features':'models/lambda models/Model_lambda_morganfeatures',
         'Avalon':'models/lambda models/Model_lambda_avalon'}

fig, ax = plt.subplots(3, 3, figsize=(10,10), dpi=300, facecolor="white")

for i, dtype in enumerate(["CircuS", "ChyLine", "Avalon", "Morgan", 'Morgan features', 'AtomPairs', 
                           "RDKFP", "RDKFP linear", 'RDKFP layered']):
    d = dtype.lower()

    best = pd.read_table(names[dtype]+'/trials.best', sep=' ').iloc[0].trial
    cv_res = pd.read_table(names[dtype]+"/trial."+str(best)+"/predictions", sep=' ')
    a = cv_res["lambdaa.observed"]
    b = cv_res[['lambdaa.predicted.repeat'+str(i) for i in range(1,11)]]
    
    ax[i//3, i%3].errorbar(a,b.mean(axis=1),
                           b.std(axis=1), fmt=".", color=colors[0], alpha=0.5)
    
    ax[i//3, i%3].plot([a.min(), a.max()], [a.min(), a.max()], "k--")
    ax[i//3, i%3].set_xlabel(r"Observed $\lambda_{max}$, nm")
    ax[i//3, i%3].set_ylabel(r"Predicted $\lambda_{max}$, nm")
    ax[i//3, i%3].set_title(dtype)
    textstr = "\n".join([
        "RMSE(CV) = %.3f"  % (rmse(a,b.mean(axis=1)), ),
        "R2(CV) = %.3f"  % (r2(a,b.mean(axis=1)), )
            ])
    ax[i//3, i%3].text(0.05, 0.95, textstr, transform=ax[i//3, i%3].transAxes,
                       fontsize=9, color=colors[0],
                       verticalalignment="top", horizontalalignment="left",
                       bbox={"boxstyle":"round", "facecolor":"white", "alpha":0})
    

plt.tight_layout(pad=2)
#plt.savefig('SI - all Lambda models.png')

In [ ]:
names = {'RDKFP':'models/lambda models/Model_lambda_rdkfp',
         'RDKFP layered':'models/lambda models/Model_lambda_layered',
         'ChyLine':'models/lambda models/Model_lambda_chyline',
         'Morgan':'models/lambda models/Model_lambda_morgan',
         'CircuS':'models/lambda models/Model_lambda_circus',
         'RDKFP linear':'models/lambda models/Model_lambda_rdkfplinear',
         'AtomPairs':'models/lambda models/Model_lambda_atompairs',
         'Morgan features':'models/lambda models/Model_lambda_morganfeatures',
         'Avalon':'models/lambda models/Model_lambda_avalon'}

fig = plt.figure(figsize = (10,4.8), dpi=300, facecolor="w")

ax1 = plt.subplot2grid((1, 3), (0, 0), colspan=2)
ax2 = plt.subplot2grid((1, 3), (0, 2))

cv_r2_results = []
for i, dtype in enumerate(["CircuS", "ChyLine", "Avalon", "Morgan", 'Morgan features', 'AtomPairs', 
                           "RDKFP", "RDKFP linear", 'RDKFP layered']):
    d = dtype.lower()
    cv_r2_results.append([])

    best = pd.read_table(names[dtype]+'/trials.best', sep=' ').iloc[0].trial
    cv_res = pd.read_table(names[dtype]+"/trial."+str(best)+"/predictions", sep=' ')
    a = cv_res["lambdaa.observed"]
    for i in range(1,11):
        b = cv_res['lambdaa.predicted.repeat'+str(i)]
        cv_r2_results[-1].append(rmse(a,b))

bp1 = ax1.boxplot(cv_r2_results, positions=[0,1,2,3,4,5,6,7,8],
                  patch_artist=True, showmeans=True)
for median in bp1['medians']:
    median.set(color='k', linewidth=1)
for mean in bp1['means']:
    mean.set(marker='s', markerfacecolor='w', markeredgecolor='k', markersize=5)
for item in ['boxes', 'whiskers', 'fliers', 'caps']:
        plt.setp(bp1[item], color='k')
plt.setp(bp1["boxes"], facecolor=colors[0], edgecolor='k')
plt.setp(bp1["fliers"], markeredgecolor='k', markersize=3)

ax1.set_xticks([0,1,2,3,4,5,6,7,8])
ax1.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)
ax1.set_xticklabels(["CircuS", "ChyLine", "Avalon", "Morgan", 'Morgan features', 'AtomPairs', 
                           "RDKFP", "RDKFP linear", 'RDKFP layered'], 
                    rotation=-45, fontsize=14,
                    ha="right",rotation_mode="anchor")
ax1.set_ylabel(r"$RMSE$, nm", fontsize=14)#, rotation=0, labelpad=-15, loc="top")
#ax.set_ylim([0.75, 0.95])
ax1.grid(color="lightgrey", axis="y", alpha=.5)
ax1.text(0.5, -0.15, 'Descriptor benchmark', transform=ax1.transAxes,
                       fontsize=14, color='k',
                       verticalalignment="top", horizontalalignment="center")

best = pd.read_table(names['CircuS']+'/trials.best', sep=' ').iloc[0].trial
cv_res = pd.read_table(names['CircuS']+"/trial."+str(best)+"/predictions", sep=' ')
a = cv_res["lambdaa.observed"]
b = cv_res[['lambdaa.predicted.repeat'+str(i) for i in range(1,11)]]
    
ax2.errorbar(a,b.mean(axis=1), b.std(axis=1), fmt=".", color=colors[0], alpha=0.5, label="CV")
    
ax2.plot([a.min(), a.max()], [a.min(), a.max()], "k--")

ext_preds = pd.read_table("Model files/SVR_lambda/predictions_lambda_bydesc.csv", sep=",")

ax2.plot(ext_preds["observed"], ext_preds["predicted_circus"], 'o', color=colors[1], markersize=4,  zorder=10, label="test")
ax2.set_xlabel(r"Observed $\lambda_{max}$, nm", fontsize=14)
ax2.set_ylabel(r"Predicted $\lambda_{max}$, nm", fontsize=14)
ax2.set_title('Cross-validation\nconsensus and external\nvalidation results\nfor CircuS fragments', fontsize=14)
#textstr = r"$RMSE$(CV) = " + str(round(rmse(a,b.mean(axis=1)),2)) + '\n' + 
textstr = r"$R^2$(CV) = " + str(round(r2(a,b.mean(axis=1)), 3))
textstr += '\n' + r"$R^2$(test) = " + str(round(r2(ext_preds["observed"], ext_preds["predicted_circus"]), 3))
ax2.text(0.05, 0.95, textstr, transform=ax2.transAxes,
                       fontsize=11, color='k',
                       verticalalignment="top", horizontalalignment="left",
                       bbox={"boxstyle":"round", "facecolor":"white", "alpha":0})

ax2.legend(loc=4)
plt.tight_layout(pad=2)
#plt.savefig('SI - Benchmark and best model for lambda.png', dpi=300)

# Best models for predictions of log$t_{1/2}$

Similar to previous, the code here reads the benchmark results for models predictiong half-life and reconstructs the plots. Random forest or XGBoost models can also be recreated by changing the folder names (adding RFR or XG in the end for all of them emtries in the 'names' variable).

In [ ]:
names = {'RDKFP':'models/logt12 models/Model_logt12_rdkfp',
         'RDKFP layered':'models/logt12 models/Model_logt12_layered',
         'ChyLine':'models/logt12 models/Model_logt12_chyline',
         'Morgan':'models/logt12 models/Model_logt12_morgan',
         'CircuS':'models/logt12 models/Model_logt12_circus',
         'RDKFP linear':'models/logt12 models/Model_logt12_rdkfplinear',
         'AtomPairs':'models/logt12 models/Model_logt12_atompairs',
         'Morgan features':'models/logt12 models/Model_logt12_morganfeatures',
         'Avalon':'models/logt12 models/Model_logt12_avalon'}

fig, ax = plt.subplots(3, 3, figsize=(10,10), dpi=300, facecolor="white")

for i, dtype in enumerate(["CircuS", "ChyLine", "Avalon", "Morgan", 'Morgan features', 'AtomPairs', 
                           "RDKFP", "RDKFP linear", 'RDKFP layered']):
    d = dtype.lower()

    best = pd.read_table(names[dtype]+'/trials.best', sep=' ').iloc[0].trial
    cv_res = pd.read_table(names[dtype]+"/trial."+str(best)+"/predictions", sep=' ')
    a = cv_res["Htime.observed"]
    b = cv_res[['Htime.predicted.repeat'+str(i) for i in range(1,11)]]

    fdata_logt12 = fdata[pd.notnull(fdata.logt12)]
    fdata_logt12 = fdata_logt12.reset_index()
    ind_switches = fdata_logt12[fdata_logt12.subset == 'switches'].index
    ind_double = fdata_logt12[fdata_logt12.subset == 'double_prop'].index

    ax[i//3, i%3].errorbar(a,b.mean(axis=1),
                           b.std(axis=1), fmt=".", color=colors[0], alpha=0.5)
    
    ax[i//3, i%3].plot([a.min(), a.max()], [a.min(), a.max()], "k--")
    ax[i//3, i%3].set_xlabel(r"Observed $\rm{log} t_{1/2}$")
    ax[i//3, i%3].set_ylabel(r"Predicted $\rm{log} t_{1/2}$")
    ax[i//3, i%3].set_title(dtype)

    textstr = "\n".join([
        "RMSE(CV) = %.3f"  % (rmse(a,b.mean(axis=1)), ),
        "R2(CV) = %.3f"  % (r2(a,b.mean(axis=1)), )
            ])
    ax[i//3, i%3].text(0.05, 0.95, textstr, transform=ax[i//3, i%3].transAxes,
                       fontsize=9, color='red',
                       verticalalignment="top", horizontalalignment="left",
                       bbox={"boxstyle":"round", "facecolor":"white", "alpha":0})
    
plt.tight_layout(pad=2)
#plt.savefig('SI - logt12 models.png')

# Summary plot for models predicting log$t_{1/2}$

In [ ]:
names = {'RDKFP':'models/logt12 models/Model_logt12_rdkfp',
         'RDKFP layered':'models/logt12 models/Model_logt12_layered',
         'ChyLine':'models/logt12 models/Model_logt12_chyline',
         'Morgan':'models/logt12 models/Model_logt12_morgan',
         'CircuS':'models/logt12 models/Model_logt12_circus',
         'RDKFP linear':'models/logt12 models/Model_logt12_rdkfplinear',
         'AtomPairs':'models/logt12 models/Model_logt12_atompairs',
         'Morgan features':'models/logt12 models/Model_logt12_morganfeatures',
         'Avalon':'models/logt12 models/Model_logt12_avalon'}

fig = plt.figure(figsize = (10,4.8), dpi=300, facecolor="w")

ax1 = plt.subplot2grid((1, 3), (0, 0), colspan=2)
ax2 = plt.subplot2grid((1, 3), (0, 2))

#fig, ax = plt.subplots(1,3, figsize = (13.4,4.5), dpi=300, facecolor="w")

cv_r2_results = []
for i, dtype in enumerate(["CircuS", "ChyLine", "Avalon", "Morgan", 'Morgan features', 'AtomPairs', 
                           "RDKFP", "RDKFP linear", 'RDKFP layered']):
    d = dtype.lower()
    cv_r2_results.append([])

    best = pd.read_table(names[dtype]+'/trials.best', sep=' ').iloc[0].trial
    cv_res = pd.read_table(names[dtype]+"/trial."+str(best)+"/predictions", sep=' ')
    a = cv_res["Htime.observed"]
    for i in range(1,11):
        b = cv_res['Htime.predicted.repeat'+str(i)]
        cv_r2_results[-1].append(rmse(a,b))

bp1 = ax1.boxplot(cv_r2_results, positions=[0,1,2,3,4,5,6,7,8],
                  patch_artist=True, showmeans=True)
for median in bp1['medians']:
    median.set(color='k', linewidth=1)
for mean in bp1['means']:
    mean.set(marker='s', markerfacecolor='w', markeredgecolor='k', markersize=5)
for item in ['boxes', 'whiskers', 'fliers', 'caps']:
        plt.setp(bp1[item], color='k')
plt.setp(bp1["boxes"], facecolor=colors[0], edgecolor='k')
plt.setp(bp1["fliers"], markeredgecolor='k', markersize=3)

ax1.set_xticks([0,1,2,3,4,5,6,7,8])
ax1.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)
ax1.set_xticklabels(["CircuS", "ChyLine", "Avalon", "Morgan", 'Morgan features', 'AtomPairs', 
                           "RDKFP", "RDKFP linear", 'RDKFP layered'], 
                    rotation=-45, fontsize=14,
                    ha="right",rotation_mode="anchor")
ax1.set_ylabel(r"$RMSE$", fontsize=14)
#ax.set_ylim([0.75, 0.95])
ax1.grid(color="lightgrey", axis="y", alpha=.5)
ax1.text(0.5, -0.15, 'Descriptor benchmark', transform=ax1.transAxes,
                       fontsize=14, color='k',
                       verticalalignment="top", horizontalalignment="center")

best = pd.read_table(names['ChyLine']+'/trials.best', sep=' ').iloc[0].trial
cv_res = pd.read_table(names['ChyLine']+"/trial."+str(best)+"/predictions", sep=' ')
a = cv_res["Htime.observed"]
b = cv_res[['Htime.predicted.repeat'+str(i) for i in range(1,11)]]

ext_preds = pd.read_table("best_models/SVR_Htime/predictions_Htime_bydescSVR.csv", sep=",")
ax2.plot(ext_preds["observed"], ext_preds["predicted_chyline"], 'o', color=colors[1], markersize=4,  zorder=10, label="test")
    
ax2.errorbar(a,b.mean(axis=1), b.std(axis=1), fmt=".", color=colors[0], alpha=0.5, label="CV")
    
ax2.plot([a.min(), a.max()], [a.min(), a.max()], "k--")
ax2.set_xlabel(r"Observed log$t_{1/2}$", fontsize=14)
ax2.set_ylabel(r"Predicted log$t_{1/2}$", fontsize=14)
ax2.set_title('Cross-validation\nconsensus and external\nvalidation results\nfor ChyLine fragments', fontsize=14)
#textstr = r"$RMSE$(CV) = " + str(round(rmse(a,b.mean(axis=1)),2)) + '\n' + 
textstr = r"$R^2$(CV) = " + str(round(r2(a,b.mean(axis=1)), 3)) + "\n" 
textstr += r"$R^2$(test) = " + str(round(r2(ext_preds["observed"], ext_preds["predicted_chyline"]), 3))
ax2.text(0.05, 0.95, textstr, transform=ax2.transAxes,
                       fontsize=11, color='k',
                       verticalalignment="top", horizontalalignment="left",
                       bbox={"boxstyle":"round", "facecolor":"white", "alpha":0})
ax2.legend(loc=4)

plt.tight_layout(pad=2)
#plt.savefig('SI - Benchmark and best models for logt12.png', dpi=300)

In [ ]:
ext_preds = pd.read_table("best_models/SVR_Htime/predictions_Htime_bydescSVR.csv", sep=",")

obs = ext_preds["observed"]
pred_chyline = ext_preds["predicted_chyline"]
pred_layered = ext_preds["predicted_layered"]

#print(rmse(obs, ext_preds[["predicted_chyline", "predicted_layered"]].mean(axis=1)), 
#    r2(obs, ext_preds[["predicted_chyline", "predicted_layered"]].mean(axis=1)))

fig, ax = plt.subplots(1, 3, figsize=(8,3.5), dpi=300)
ax[0].plot([obs.min(), obs.max()], [obs.min(), obs.max()], "k--")
rms = rmse(ext_preds["observed"], ext_preds["predicted_chyline"])
ax[0].plot(ext_preds["observed"], ext_preds["predicted_chyline"], 'o', color=colors[1], markersize=4, 
        label="Only Chyline, "+str(np.round(rms, 3)))
textstr = r"$RMSE$(test) = " + str(round(rms, 3)) + "\n" 
textstr += r"$R^2$(test) = " + str(round(r2(ext_preds["observed"], ext_preds["predicted_chyline"]), 3))
ax[0].text(0.05, 0.95, textstr, transform=ax[0].transAxes,
                       fontsize=9, color='k',
                       verticalalignment="top", horizontalalignment="left",
                       bbox={"boxstyle":"round", "facecolor":"white", "alpha":0})

rms = rmse(ext_preds["observed"], ext_preds[["predicted_chyline", "predicted_layered"]].mean(axis=1))
ax[1].plot([obs.min(), obs.max()], [obs.min(), obs.max()], "k--")
ax[1].plot(ext_preds["observed"], ext_preds[["predicted_chyline", "predicted_layered"]].mean(axis=1), 'o', 
            color=colors[1], markersize=4, label="Chyline+layered, "+str(np.round(rms, 3)))
improve = np.abs(ext_preds["observed"]-ext_preds["predicted_chyline"])< np.abs(ext_preds["observed"]-ext_preds[["predicted_chyline", "predicted_layered"]].mean(axis=1))
ax[1].vlines(ext_preds["observed"], ymin=ext_preds["predicted_chyline"], 
             ymax=ext_preds[["predicted_chyline", "predicted_layered"]].mean(axis=1),
             colors=[colors[0] if i else colors[2] for i in improve ])
textstr = r"$RMSE$(test) = " + str(round(rms, 3)) + "\n" 
textstr += r"$R^2$(test) = " + str(round(r2(ext_preds["observed"], ext_preds[["predicted_chyline", "predicted_layered"]].mean(axis=1)), 3))
ax[1].text(0.05, 0.95, textstr, transform=ax[1].transAxes,
                       fontsize=9, color='k',
                       verticalalignment="top", horizontalalignment="left",
                       bbox={"boxstyle":"round", "facecolor":"white", "alpha":0})


rms = rmse(ext_preds["observed"], ext_preds[["predicted_chyline", "predicted_rdkfp", "predicted_layered"]].mean(axis=1))
ax[2].plot([obs.min(), obs.max()], [obs.min(), obs.max()], "k--")
ax[2].plot(ext_preds["observed"], ext_preds[["predicted_chyline", "predicted_rdkfp", "predicted_layered"]].mean(axis=1), 'o', 
           color=colors[1], markersize=4, label="Chyline+RDK linear+layered, "+str(np.round(rms, 3)))
improve = np.abs(ext_preds["observed"]-ext_preds["predicted_chyline"])< np.abs(ext_preds["observed"]-ext_preds[["predicted_chyline", "predicted_rdkfplinear", "predicted_layered"]].mean(axis=1))
ax[2].vlines(ext_preds["observed"], ymin=ext_preds["predicted_chyline"], 
             ymax=ext_preds[["predicted_chyline", "predicted_rdkfp", "predicted_layered"]].mean(axis=1),
             colors=[colors[0] if i else colors[2] for i in improve ])
textstr = r"$RMSE$(test) = " + str(round(rms, 3)) + "\n" 
textstr += r"$R^2$(test) = " + str(round(r2(ext_preds["observed"], 
                                            ext_preds[["predicted_chyline", "predicted_rdkfp", "predicted_layered"]].mean(axis=1)),
                                            3))
ax[2].text(0.05, 0.95, textstr, transform=ax[2].transAxes,
                       fontsize=9, color='k',
                       verticalalignment="top", horizontalalignment="left",
                       bbox={"boxstyle":"round", "facecolor":"white", "alpha":0})

ax[0].set_xlabel(r"Observed log$t_{1/2}$", fontsize=12)
ax[0].set_ylabel(r"Predicted log$t_{1/2}$", fontsize=12)
ax[0].set_title("Predictions by model built\nonly on ChyLine fragments", fontsize=11)
ax[1].set_xlabel(r"Observed log$t_{1/2}$", fontsize=12)
ax[1].set_ylabel(r"Predicted log$t_{1/2}$", fontsize=12)
ax[1].set_title("Predictions by consensus\nmodel built on ChyLine\nand RDKit layered FP", fontsize=11)
ax[2].set_xlabel(r"Observed log$t_{1/2}$", fontsize=12)
ax[2].set_ylabel(r"Predicted log$t_{1/2}$", fontsize=12)
ax[2].set_title("Predictions by consensus\nmodel built on ChyLine,\nRDKit FP and layered FP", fontsize=11)
plt.tight_layout(pad=2)
#plt.savefig("consensus modeling logt12.png", dpi=300)

In [ ]:
rms = rmse(ext_preds["observed"], ext_preds["predicted_chyline"])
rr = r2(ext_preds["observed"], ext_preds["predicted_chyline"])
print("Base", rms, rr)

for c in ['predicted_torsion', 'predicted_layered',
       'predicted_circus', 'predicted_morgan', 'predicted_rdkfp',
       'predicted_atompairs', 'predicted_avalon',
       'predicted_morganfeatures', 'predicted_rdkfplinear']:
    rms = rmse(ext_preds["observed"], ext_preds[["predicted_chyline", c]].mean(axis=1))
    rr = r2(ext_preds["observed"], ext_preds[["predicted_chyline", c]].mean(axis=1))
    print(c, rms, rr)

print()
for c in ['predicted_torsion', 'predicted_layered',
       'predicted_circus', 'predicted_morgan', 
       'predicted_atompairs', 'predicted_avalon',
       'predicted_morganfeatures', 'predicted_rdkfplinear']:
    rms = rmse(ext_preds["observed"], ext_preds[["predicted_chyline", 'predicted_rdkfp', c]].mean(axis=1))
    rr = r2(ext_preds["observed"], ext_preds[["predicted_chyline", 'predicted_rdkfp', c]].mean(axis=1))
    print(c, rms, rr)

## External predictions by best models

The following code is for reading, retraining and reapplying the best model obtained in the benchmark for each descriptor type.
The results of the predicitons are already available in the correspoiding folder as a CSV, so technically this code is not required, it is given to demonstrate how a saved model may be reused. Please note that the scikit-learn version 1.2.2 was used for training the models initially, and using any other version may lead to slightly different results.

In [ ]:
names_models = {'rdkfp':'Model files/SVR_lambda/SVR_trial396_Rdkfp_1024_4.pkl',
         'layered':'Model files/SVR_lambda/SVR_trial497_Layered_1024_6.pkl',
         'chyline':'Model files/SVR_lambda/SVR_trial24_Chyline_2_8.pkl',
         'morgan':'Model files/SVR_lambda/SVR_trial421_Morgan_1024_2.pkl',
         'circus':'Model files/SVR_lambda/SVR_trial126_Circus_0_2.pkl',
         'rdkfplinear':'Model files/SVR_lambda/SVR_trial335_Rdkfplinear_1024_8.pkl',
         'atompairs':'Model files/SVR_lambda/SVR_trial320_Atompairs_1024.pkl',
         'morganfeatures':'Model files/SVR_lambda/SVR_trial244_Morganfeatures_1024_3.pkl',
         'avalon':'Model files/SVR_lambda/SVR_trial197_Avalon_1024.pkl'}

predictions_table = pd.DataFrame(columns=["observed"])
predictions_table["observed"] = np.array(fdata_lambda.loc[test_lambda]["lambda"])

for i, dtype in enumerate(["circus", "chyline", "avalon", "morgan", 'morganfeatures', 'atompairs', 
                           "rdkfp", "rdkfplinear", 'layered']):
    with open(names_models[dtype], "rb") as f:
        model = pickle.load(f)
    model.fit(fdata_lambda.loc[train_lambda]["mol"], fdata_lambda.loc[train_lambda]["lambda"])
    preds = model.predict(fdata_lambda.loc[test_lambda]["mol"])
    predictions_table["predicted_"+dtype] = preds
    print(dtype, r2(fdata_lambda.loc[test_lambda]["lambda"], preds))

predictions_table

## Comparison of methods in the benchmark

In [ ]:
names_svm = {'RDKFP':'All_models/Model_lambda_rdkfp',
         'RDKFP layered':'All_models/Model_lambda_layered',
         'ChyLine':'All_models/Model_lambda_chyline',
         'Morgan':'All_models/Model_lambda_morgan',
         'CircuS':'All_models/Model_lambda_circus',
         'RDKFP linear':'All_models/Model_lambda_rdkfplinear',
         'AtomPairs':'All_models/Model_lambda_atompairs',
         'Morgan features':'All_models/Model_lambda_morganfeatures',
         'Avalon':'All_models/Model_lambda_avalon'}

names_rfr = {'RDKFP':'All_models/Model_lambda_rdkfpRFR',
         'RDKFP layered':'All_models/Model_lambda_layeredRFR',
         'ChyLine':'All_models/Model_lambda_chylineRFR',
         'Morgan':'All_models/Model_lambda_morganRFR',
         'CircuS':'All_models/Model_lambda_circusRFR',
         'RDKFP linear':'All_models/Model_lambda_rdkfplinearRFR',
         'AtomPairs':'All_models/Model_lambda_atompairsRFR',
         'Morgan features':'All_models/Model_lambda_morganfeaturesRFR',
         'Avalon':'All_models/Model_lambda_avalonRFR'}

names_xgb = {'RDKFP':'All_models/Model_lambda_rdkfpXG',
         'RDKFP layered':'All_models/Model_lambda_layeredXG',
         'ChyLine':'All_models/Model_lambda_chylineXG',
         'Morgan':'All_models/Model_lambda_morganXG',
         'CircuS':'All_models/Model_lambda_circusXG',
         'RDKFP linear':'All_models/Model_lambda_rdkfplinearXG',
         'AtomPairs':'All_models/Model_lambda_atompairsXG',
         'Morgan features':'All_models/Model_lambda_morganfeaturesXG',
         'Avalon':'All_models/Model_lambda_avalonXG'}

r2_svm = []
r2_rfr = []
r2_xgb = []

for i, dtype in enumerate(["CircuS", "ChyLine", "Avalon", "Morgan", 'Morgan features', 'AtomPairs', 
                           "RDKFP", "RDKFP linear", 'RDKFP layered']):
    d = dtype.lower()

    best = pd.read_table(names_svm[dtype]+'/trials.best', sep=' ').iloc[0].trial
    cv_res = pd.read_table(names_svm[dtype]+"/trial."+str(best)+"/predictions", sep=' ')
    a = cv_res["lambdaa.observed"]
    b = cv_res[['lambdaa.predicted.repeat'+str(i) for i in range(1,11)]]
    r2_svm.append(r2(a, b.mean(axis=1))) 

    best = pd.read_table(names_rfr[dtype]+'/trials.best', sep=' ').iloc[0].trial
    cv_res = pd.read_table(names_rfr[dtype]+"/trial."+str(best)+"/predictions", sep=' ')
    a = cv_res["lambdaa.observed"]
    b = cv_res[['lambdaa.predicted.repeat'+str(i) for i in range(1,11)]]
    r2_rfr.append(r2(a, b.mean(axis=1))) 

    best = pd.read_table(names_xgb[dtype]+'/trials.best', sep=' ').iloc[0].trial
    cv_res = pd.read_table(names_xgb[dtype]+"/trial."+str(best)+"/predictions", sep=' ')
    a = cv_res["lambdaa.observed"]
    b = cv_res[['lambdaa.predicted.repeat'+str(i) for i in range(1,11)]]
    r2_xgb.append(r2(a, b.mean(axis=1))) 

fig, ax = plt.subplots(figsize = (7,4), dpi=300, facecolor="w")
ax.plot([0,1,2,3,4,5,6,7,8], r2_svm, "o-", color=colors[0], label="SVM")
ax.plot([0,1,2,3,4,5,6,7,8], r2_rfr, "o-", color=colors[1], label="RF")
ax.plot([0,1,2,3,4,5,6,7,8], r2_xgb, "o-", color=colors[2], label="XGB")

ax.set_xticks([0,1,2,3,4,5,6,7,8])
ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)
ax.set_xticklabels(["CircuS", "ChyLine", "Avalon", "Morgan", 'Morgan features', 'AtomPairs', 
                           "RDKFP", "RDKFP linear", 'RDKFP layered'], 
                    rotation=-45, fontsize=14,
                    ha="right",rotation_mode="anchor")
ax.set_ylabel(r"$R^2$", fontsize=14, rotation=0, labelpad=15)#, loc="top")
#ax.set_ylim([0.75, 0.95])
ax.grid(color="lightgrey", axis="y", alpha=.5)
plt.legend()
plt.tight_layout()
#plt.savefig("comparison of methods on lambda.png", dpi=300)

# Visualization of chemical space

The following code reproduces the UMAP visualization of the chemical space for both property datasets. Please note that UMAP is stochastic in nature, and the resulting map may not be exactly as presented in the manuscript, even when using pretrained mappers (provided in the repository).

In [ ]:
import umap
import numba
from doptools import ChythonCircus, ChythonLinear

def tanimoto_dist(a, b):
    return 1 - np.dot(a,b)/(np.dot(a,a) + np.dot(b,b) - np.dot(a,b))

lambda_fragmentor = ChythonCircus(0,2)
lambda_fragmentor.fit(fdata_lambda.loc[train_lambda]["mol"])

lambda_train_desc = lambda_fragmentor.transform(fdata_lambda.loc[train_lambda]["mol"])
lambda_test_desc = lambda_fragmentor.transform(fdata_lambda.loc[test_lambda]["mol"])

# === Following code is used for training the UMAP model. 
# === The models are pretrained in the manuscript, please find the pickle files in the repository
#lambda_umap_model = umap.UMAP(n_neighbors=25, min_dist=0.5, n_components=2, metric=tanimoto_dist)
#lambda_umap_model.fit(lambda_train_desc)
#with open('mapper.lambda.pkl', 'wb') as f:
#    pickle.dump(lambda_umap_model, f, pickle.HIGHEST_PROTOCOL)
#lambda_umap_embedding = lambda_umap_model.transform(lambda_train_desc)
#lambda_test_embedding = lambda_umap_model.transform(lambda_test_desc)



In [ ]:
with open('mapper.lambda.pkl', 'rb') as f:
    lambda_umap_model = pickle.load(f)
lambda_umap_embedding = lambda_umap_model.transform(lambda_train_desc)
lambda_test_embedding = lambda_umap_model.transform(lambda_test_desc)

plt.figure(figsize=(4.5, 4), dpi=300)
plt.plot(lambda_umap_embedding[:, 0], lambda_umap_embedding[:, 1], "o", color=colors[0], markersize=3, label='training')
plt.plot(lambda_test_embedding[:, 0], lambda_test_embedding[:, 1], "o", color=colors[1], markersize=3, label='test')
plt.plot(lambda_test_embedding[[9, 26, 30, 35, 64], 0], lambda_test_embedding[[9, 26, 30, 35, 64], 1], "o", 
         color=colors[1], markeredgecolor="k", markersize=3)
plt.title(r'UMAP visualization of $\lambda_{max}$ chemical space')
plt.xlabel('')
plt.ylabel('')
plt.legend(title='Dataset', )
plt.tight_layout()
#plt.savefig('chemical_space_lambda.png', dpi=300)

In [ ]:
print(pd.read_table('../said_All_models/Modd_chyline/trials.best', sep=' ').iloc[0])
logt12_fragmentor = ChythonCircus(0,2)
logt12_fragmentor.fit(fdata_log.loc[train_logt12]["mol"])

logt12_train_desc = logt12_fragmentor.transform(fdata_log.loc[train_logt12]["mol"])
logt12_test_desc = logt12_fragmentor.transform(fdata_log.loc[test_logt12]["mol"])

# === Following code is used for training the UMAP model. 
# === The models are pretrained in the manuscript, please find the pickle files in the repository
#logt12_umap_model = umap.UMAP(n_neighbors=25, min_dist=0.5, n_components=2, metric=tanimoto_dist)
#logt12_umap_model.fit(logt12_train_desc)
#with open('mapper.logt12.pkl', 'wb') as f:
#    pickle.dump(logt12_umap_model, f, pickle.HIGHEST_PROTOCOL)
#logt12_umap_embedding = logt12_umap_model.transform(logt12_train_desc)
#logt12_test_embedding = logt12_umap_model.transform(logt12_test_desc)



In [ ]:
with open('mapper.logt12.pkl', 'rb') as f:
    logt12_umap_model = pickle.load(f)
logt12_umap_embedding = logt12_umap_model.transform(logt12_train_desc)
logt12_test_embedding = logt12_umap_model.transform(logt12_test_desc)

plt.figure(figsize=(4.5, 4), dpi=300)
plt.plot(logt12_umap_embedding[:, 0], logt12_umap_embedding[:, 1], "o", color=colors[0], markersize=4, label='training')
plt.plot(logt12_test_embedding[:, 0], logt12_test_embedding[:, 1], "o", color=colors[1], markersize=4, label='test')
plt.plot(logt12_test_embedding[[1,4,8,12,13], 0], logt12_test_embedding[[1,4,8,12,13], 1], "o", 
         color=colors[1], markeredgecolor="k", markersize=4)
plt.title(r'UMAP visualization of log$t_{1/2}$ chemical space')
plt.xlabel('')
plt.ylabel('')
plt.legend(title='Dataset', )
plt.tight_layout()
#plt.savefig('chemical_space_logt12.png', dpi=300)

# Model interpretation

The following code visulaizes the ColorAtom representations for the model predicting the half-life.

In [ ]:
from doptools import ColorAtom

ca = ColorAtom()

with open("../best_models/SVR_Htime/SVR_trial57_chyline_2_6.pkl", "rb") as f:
    model = pickle.load(f)
model.fit(fdata_log.loc[train_logt12]["mol"], fdata_log.loc[train_logt12]["logt12"])

ca.set_pipeline(model)

ca.output_html(fdata_log["mol"].loc[test_logt12[13]])